In [1]:
import numpy as np, pandas as pd
import os, phate, anndata, scprep
import pickle, tqdm
from glob import glob
import stream as st
import warnings

import kmapper as km
from sklearn import preprocessing
from kmap_sup import GM_extend, KM_extend
from sklearn.cluster import KMeans, DBSCAN
from sklearn import preprocessing
import math
from flow_util import Flow_Util
from flowio.create_fcs import create_fcs
import flowutils

from tqdm import tqdm
from scipy.spatial import ConvexHull
from scipy.spatial import Delaunay
from pyhull.convex_hull import ConvexHull as convexhull1
from skimage.draw import ellipse
from skimage.measure import find_contours, approximate_polygon, subdivide_polygon
import itertools
import xgboost as xgb
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import operator

from IPython.display import display
pd.options.display.max_columns = None

%load_ext autoreload
%autoreload 2
warnings.filterwarnings('ignore')
phate_operator = phate.PHATE(n_jobs=15)

fut = Flow_Util()

In [2]:
dat_path = "../../data/human data/data_15072020/"
fin = []

df_xls = pd.read_excel(dat_path+"20200715sortForDataTransferToAn.xlsx",sheet_name=0)
col_new = [i.split(" ::")[0] for i in df_xls.columns]
df_xls.columns = col_new
dep_col = ["FJ"+i if "Comp" in i else i for i in col_new]
df_xls.columns = dep_col
dep_col.remove('FJComp-PE-A')
full_col = list(df_xls.columns)
df_xls["id"] = 1
fin.append(df_xls[full_col +["id"]])

df_xls = pd.read_excel(dat_path+"20200715sortForDataTransferToAn.xlsx",sheet_name=1)
df_xls.columns = full_col
df_xls["id"] = 2
fin.append(df_xls[full_col +["id"]])

df_xls = pd.read_excel(dat_path+"20200715sortForDataTransferToAn.xlsx",sheet_name=2)
df_xls.columns = full_col
df_xls["id"] = 3
fin.append(df_xls[full_col +["id"]])

df_xls = pd.read_excel(dat_path+"20200715sortForDataTransferToAn.xlsx",sheet_name=3)
df_xls.columns = full_col
df_xls["id"] = 4
fin.append(df_xls[full_col +["id"]])

dat_path = "../../data/human data/data_extracted/"

fin = pd.concat(fin)

fin.shape

(831, 15)

In [3]:
save_path = "../../output/human_screen/heat_map_assemble/"
df_or = pd.read_csv(save_path + "human_screen_full.csv")
df_or = df_or[df_or.columns[1:]]
df_or.shape

(198858, 397)

In [25]:
# match all screening parameter into the reference data
from numba import njit
from sklearn.neighbors import NearestNeighbors
@njit(parallel=True)
def get_list(k):
    return [[h-831 for h in k[x] if h>831][:10] for x in range(k.shape[0])]

def get_assemble_v2(df , df_or, name_list):
    #df = df.loc[df["FJComp-BUV395-A"] > 200]
    df["id"] = 0

    df_out = pd.concat([df_or[dep_col+["id"]],df[dep_col+["id"]]])
    
    X_input = preprocessing.scale(df_out[dep_col])
    phate_operator = phate.PHATE(n_jobs=-2,knn=50)
    Y_phate = phate_operator.fit_transform(X_input)
    phate_operator.set_params(n_components=10)
    Y_phate_3d = phate_operator.transform()
    
    for i in range(10):
        df_out["phate_"+str(i)] = Y_phate_3d[:,i]
        
    knn = NearestNeighbors(n_neighbors=5000)
    phate_list = ["phate_"+str(i) for i in range(10)]
    knn.fit(df_out[phate_list].values)
    temp1 = df_out.loc[df_out.id!=0]
    temp2 = knn.kneighbors(temp1[phate_list].values, return_distance=False)
    #check = df_out["FJComp-PE-A"].values
    
    t1 = get_list(temp2)
    for name_col in name_list:
        check=df[name_col].values
        t2 = [check[i].mean() for i in t1]
        df_or[name_col] = t2

In [26]:
name_list = list(df_or.columns)[13:]
dep_col = list(dep_col)

In [27]:
get_assemble_v2(df_or, fin, name_list)

Calculating PHATE...
  Running PHATE on 199689 cells and 13 genes.
  Calculating graph and diffusion operator...
    Calculating KNN search...
    Calculated KNN search in 682.43 seconds.
    Calculating affinities...
    Calculated affinities in 31.51 seconds.
  Calculated graph and diffusion operator in 717.46 seconds.
  Calculating landmark operator...
    Calculating SVD...
    Calculated SVD in 226.33 seconds.
    Calculating KMeans...
    Calculated KMeans in 63.27 seconds.
  Calculated landmark operator in 295.02 seconds.
  Calculating optimal t...
    Automatically selected t = 22
  Calculated optimal t in 1.46 seconds.
  Calculating diffusion potential...
  Calculated diffusion potential in 0.43 seconds.
  Calculating metric MDS...
  Calculated metric MDS in 14.72 seconds.
Calculated PHATE in 1029.14 seconds.
Calculating metric MDS...
Calculated metric MDS in 115.78 seconds.


In [28]:
fin.head()

,FSC-A,FSC-H,FSC-W,SSC-A,SSC-H,SSC-W,FJComp-APC-A,FJComp-APC-Cy7-A,FJComp-BUV395-A,FJComp-BV421-A,FJComp-BV711-A,FJComp-FITC-A,FJComp-PerCP-Cy5-5-A,screen_p1_426,screen_p1_428,screen_p1_430,screen_p1_432,screen_p1_434,screen_p1_436,screen_p1_438,screen_p1_440,screen_p1_442,screen_p1_444,screen_p1_446,screen_p1_448,screen_p1_450,screen_p1_452,screen_p1_454,screen_p1_456,screen_p1_458,screen_p1_460,screen_p1_462,screen_p1_464,screen_p1_466,screen_p1_468,screen_p1_470,screen_p1_472,screen_p1_474,screen_p1_476,screen_p1_478,screen_p1_480,screen_p1_482,screen_p1_484,screen_p1_486,screen_p1_488,screen_p1_490,screen_p1_492,screen_p1_494,screen_p1_496,screen_p1_498,screen_p1_500,screen_p1_502,screen_p1_504,screen_p1_506,screen_p1_508,screen_p1_510,screen_p1_512,screen_p1_514,screen_p1_516,screen_p1_518,screen_p1_520,screen_p1_522,screen_p1_524,screen_p1_526,screen_p1_528,screen_p1_530,screen_p1_532,screen_p1_534,screen_p1_536,screen_p1_538,screen_p1_540,screen_p1_542,screen_p1_544,screen_p1_546,screen_p1_548,screen_p1_550,screen_p1_552,screen_p1_554,screen_p1_556,screen_p1_558,screen_p1_560,screen_p1_562,screen_p1_564,screen_p1_566,screen_p1_568,screen_p1_570,screen_p1_572,screen_p1_574,screen_p1_576,screen_p1_578,screen_p1_580,screen_p1_582,screen_p1_584,screen_p1_586,screen_p1_588,screen_p1_590,screen_p1_592,screen_p1_594,screen_p1_596,screen_p1_598,screen_p1_600,screen_p1_602,screen_p1_604,screen_p1_606,screen_p1_608,screen_p1_610,screen_p1_612,screen_p1_614,screen_p1_616,screen_p2_306,screen_p2_308,screen_p2_310,screen_p2_312,screen_p2_314,screen_p2_316,screen_p2_318,screen_p2_320,screen_p2_322,screen_p2_324,screen_p2_326,screen_p2_328,screen_p2_330,screen_p2_332,screen_p2_334,screen_p2_336,screen_p2_338,screen_p2_340,screen_p2_342,screen_p2_344,screen_p2_346,screen_p2_348,screen_p2_350,screen_p2_352,screen_p2_354,screen_p2_356,screen_p2_358,screen_p2_360,screen_p2_362,screen_p2_364,screen_p2_366,screen_p2_368,screen_p2_370,screen_p2_372,screen_p2_374,screen_p2_376,screen_p2_378,screen_p2_380,screen_p2_382,screen_p2_384,screen_p2_386,screen_p2_388,screen_p2_390,screen_p2_392,screen_p2_394,screen_p2_396,screen_p2_398,screen_p2_400,screen_p2_402,screen_p2_404,screen_p2_406,screen_p2_408,screen_p2_410,screen_p2_412,screen_p2_414,screen_p2_416,screen_p2_418,screen_p2_420,screen_p2_422,screen_p2_424,screen_p2_426,screen_p2_428,screen_p2_430,screen_p2_432,screen_p2_434,screen_p2_436,screen_p2_438,screen_p2_440,screen_p2_442,screen_p2_444,screen_p2_446,screen_p2_448,screen_p2_450,screen_p2_452,screen_p2_454,screen_p2_456,screen_p2_458,screen_p2_460,screen_p2_462,screen_p2_464,screen_p2_466,screen_p2_468,screen_p2_470,screen_p2_472,screen_p2_474,screen_p2_476,screen_p2_478,screen_p2_480,screen_p2_482,screen_p2_484,screen_p2_486,screen_p2_488,screen_p2_490,screen_p2_492,screen_p2_494,screen_p2_496,screen_p3_323,screen_p3_325,screen_p3_327,screen_p3_329,screen_p3_331,screen_p3_333,screen_p3_335,screen_p3_337,screen_p3_339,screen_p3_341,screen_p3_343,screen_p3_345,screen_p3_347,screen_p3_349,screen_p3_351,screen_p3_353,screen_p3_355,screen_p3_357,screen_p3_359,screen_p3_361,screen_p3_363,screen_p3_365,screen_p3_367,screen_p3_369,screen_p3_371,screen_p3_373,screen_p3_375,screen_p3_377,screen_p3_379,screen_p3_381,screen_p3_383,screen_p3_385,screen_p3_387,screen_p3_389,screen_p3_391,screen_p3_393,screen_p3_395,screen_p3_397,screen_p3_399,screen_p3_401,screen_p3_403,screen_p3_405,screen_p3_407,screen_p3_409,screen_p3_411,screen_p3_413,screen_p3_415,screen_p3_417,screen_p3_419,screen_p3_421,screen_p3_423,screen_p3_425,screen_p3_427,screen_p3_429,screen_p3_431,screen_p3_433,screen_p3_435,screen_p3_437,screen_p3_439,screen_p3_441,screen_p3_443,screen_p3_445,screen_p3_447,screen_p3_449,screen_p3_451,screen_p3_453,screen_p3_455,screen_p3_457,screen_p3_459,screen_p3_461,screen_p3_463,screen_p3_465,screen_p3_467,screen_p3_469,screen_p3_471,screen_p3_473,screen_p3_475,screen_p3_477,screen_p3_479,screen_p3_481,screen_p3_483,screen_p3_485,screen_p3_487,sc

In [8]:
df_or["id"]=0
df_t1 = df_or.loc[df_or.screen_p1_426<200]
#df_t1 = df_t1.loc[df_t1.screen_p2_376>500]
df_t1.reset_index(inplace=True)
df_t1.drop(columns=["index"], inplace=True)
fin = fin[df_t1.columns]
df_fin = pd.concat([df_t1, fin])
df_fin.shape

(111103, 398)

In [9]:
dep_col = df_fin.columns[:-1]
len(dep_col)

397

In [13]:
dep_col[:14]

Index(['FSC-A', 'FSC-H', 'FSC-W', 'SSC-A', 'SSC-H', 'SSC-W', 'FJComp-APC-A',
       'FJComp-APC-Cy7-A', 'FJComp-BUV395-A', 'FJComp-BV421-A',
       'FJComp-BV711-A', 'FJComp-FITC-A', 'FJComp-PerCP-Cy5-5-A',
       'screen_p1_426'],
      dtype='object')

In [14]:
X_input = preprocessing.scale(df_fin[dep_col[:13]])
phate_operator = phate.PHATE(n_jobs=-2,knn=50)
Y_phate = phate_operator.fit_transform(X_input)
phate_operator.set_params(n_components=10)
Y_phate_3d = phate_operator.transform()
np.save(save_path + "phate10d_15072020.npy",Y_phate_3d)
phate_operator.set_params(n_components=3)
Y_phate_3d = phate_operator.transform()
np.save(save_path + "phate3d_15072020.npy",Y_phate_3d)

Calculating PHATE...
  Running PHATE on 111103 cells and 13 genes.
  Calculating graph and diffusion operator...
    Calculating KNN search...
    Calculated KNN search in 294.00 seconds.
    Calculating affinities...
    Calculated affinities in 19.66 seconds.
  Calculated graph and diffusion operator in 315.48 seconds.
  Calculating landmark operator...
    Calculating SVD...
    Calculated SVD in 90.42 seconds.
    Calculating KMeans...
    Calculated KMeans in 42.96 seconds.
  Calculated landmark operator in 137.23 seconds.
  Calculating optimal t...
    Automatically selected t = 20
  Calculated optimal t in 1.49 seconds.
  Calculating diffusion potential...
  Calculated diffusion potential in 0.46 seconds.
  Calculating metric MDS...
  Calculated metric MDS in 30.85 seconds.
Calculated PHATE in 485.55 seconds.
Calculating metric MDS...
Calculated metric MDS in 206.28 seconds.
Calculating metric MDS...
Calculated metric MDS in 26.18 seconds.


In [25]:
fut = Flow_Util()
fin = []
save_path = "../../data/human data/data_10062020/Location_analysis_for_An/"
ref_file = save_path + "export_83539_P7.fcs"
df = fut.read_fcs(ref_file)
full_col = list(df.columns[:-1])
dep_col = list(df.columns[:-1])
dep_col.remove('FJComp-PE-A')
df["id"] = 1
fin.append(df[full_col +["id"]])

ref_file = save_path + "export_83539_P8.fcs"
df = fut.read_fcs(ref_file)
df["id"] = 2
fin.append(df[full_col +["id"]])

ref_file = save_path + "export_83539_P9.fcs"
df = fut.read_fcs(ref_file)
df["id"] = 3
fin.append(df[full_col +["id"]])

ref_file = save_path + "export_83539_P10.fcs"
df = fut.read_fcs(ref_file)
df["id"] = 4
fin.append(df[full_col +["id"]])

ref_file = save_path + "export_83539_P11.fcs"
df = fut.read_fcs(ref_file)
df["id"] = 5
fin.append(df[full_col +["id"]])

fin = pd.concat(fin)

fin.shape

(17387, 15)

In [3]:
save_path = "../../output/human_screen/heat_map_assemble/"
df_or = pd.read_csv(save_path + "human_screen_full.csv")
df_or = df_or[df_or.columns[1:]]
df_or.shape

(198858, 397)

In [2]:
fut = Flow_Util()

df_out = []
path = "../../data/human data/hCB_Screering/Plate-1/"
save_path = "../../output/human_screen/plate_1/"
ref_file = path + "424_dens.fcs"
df = fut.read_fcs(ref_file)
full_col = fut.chan_name[:-2]
dep_col = fut.chan_name[:-2]
dep_col.remove('FJComp-PE-A')
df = df[full_col]
df = df.loc[df["FJComp-BUV395-A"] > 200]
df["id"] = 0
df_out.append(df)

path = "../../data/human data/hCB_Screering/Plate-1/"
for i in range(426,440,2):
    screen_file = path + str(i) + "_dens.fcs"
    df = fut.read_fcs(screen_file)
    df = df[full_col]
    df = df.loc[df["FJComp-BUV395-A"] > 200]
    df["id"] = 0
    df_out.append(df)

path = "../../data/human data/hCB_Screering/Plate-2/"
for i in range(304,310,2):
    screen_file = path + str(i) + "_dens.fcs"
    df = fut.read_fcs(screen_file)
    df = df[full_col]
    df = df.loc[df["FJComp-BUV395-A"] > 200]
    df["id"] = 0
    df_out.append(df)

path = "../../data/human data/hCB_Screering/Plate-3/"
for i in range(321,340,2):
    screen_file = path + str(i) + "_dens.fcs"
    df = fut.read_fcs(screen_file)
    df = df[full_col]
    df = df.loc[df["FJComp-BUV395-A"] > 200]
    df["id"] = 0
    df_out.append(df)

path = "../../data/human data/hCB_Screering/Plate-4/"
screen_file = path + "904_dens.fcs"
df = fut.read_fcs(screen_file)
df = df[full_col]
df = df.loc[df["FJComp-BUV395-A"] > 200]
df["id"] = 0
df_out.append(df)

for i in range(116,140,2):
    screen_file = path + str(i) + "_dens.fcs"
    df = fut.read_fcs(screen_file)
    df = df[full_col]
    df = df.loc[df["FJComp-BUV395-A"] > 200]
    df["id"] = 0
    df_out.append(df)
    
path = "../../data/human data/hCB/2_Panel_v2/1_Whole_cell_data/Patient_ID-026/"
for i in range(43,44,2):
    screen_file = path + str(i) + "_dens.fcs"
    df = fut.read_fcs(screen_file)
    df = df[full_col]
    df = df.loc[df["FJComp-BUV395-A"] > 200]
    df["id"] = 0
    df_out.append(df)
    
path = "../../data/human data/hCB/2_Panel_v2/1_Whole_cell_data/Patient_ID-029/"
for i in ["11"]:
    screen_file = path + i + "_dens.fcs"
    df = fut.read_fcs(screen_file)
    df = df[full_col]
    df = df.loc[df["FJComp-BUV395-A"] > 200]
    df["id"] = 0
    df_out.append(df)
    
path = "../../data/human data/hCB/2_Panel_v2/1_Whole_cell_data/Patient_ID-030/"
for i in range(55,56,2):
    screen_file = path + str(i) + "_dens.fcs"
    df = fut.read_fcs(screen_file)
    df = df[full_col]
    df = df.loc[df["FJComp-BUV395-A"] > 200]
    df["id"] = 0
    df_out.append(df)
    
path = "../../data/human data/hCB/2_Panel_v2/1_Whole_cell_data/Patient_ID-031/"
for i in range(91,92,2):
    screen_file = path + str(i) + "_dens.fcs"
    df = fut.read_fcs(screen_file)
    df = df[full_col]
    df = df.loc[df["FJComp-BUV395-A"] > 200]
    df["id"] = 0
    df_out.append(df)

fin = pd.concat(df_out)
fin.shape

(276498, 15)

In [2]:
fut = Flow_Util()
fin = []
path = "../../data/human data/hCB_Screering/Plate-1/"
save_path = "../../data/human data/data_extracted/Excel_file/20200610/fcs/"
ref_file = save_path + "20200610_BFU_E.fcs"
df = fut.read_fcs(ref_file)
re_col = ["FJ"+i if "Comp" in i else i for i in df.columns]
df.columns = re_col
full_col = list(df.columns[1:])
dep_col = list(df.columns[1:])
dep_col.remove('FJComp-PE-A')
fin.append(df[full_col])

ref_file = save_path + "20200610_CFU_GEMM.fcs"
df = fut.read_fcs(ref_file)
df.columns = re_col
fin.append(df[full_col])

ref_file = save_path + "20200610_CFU_GM.fcs"
df = fut.read_fcs(ref_file)
df.columns = re_col
fin.append(df[full_col])

fin = pd.concat(fin)

fin["id"] = 0
fin.shape

(28, 15)

In [4]:
# match all screening parameter into the reference data
from numba import njit
from sklearn.neighbors import NearestNeighbors
@njit(parallel=True)
def get_list(k):
    return [[h for h in k[x] if h>17387][:10] for x in range(k.shape[0])]

def get_assemble(path, file_no, df_or, name_col):
    df = fut.read_fcs(path + str(file_no) + "_down.fcs")
    df = df[full_col]
    #df = df.loc[df["FJComp-BUV395-A"] > 200]
    df["id"] = 1

    df_out = pd.concat([df_or[full_col+["id"]],df])
    
    X_input = preprocessing.scale(df_out[dep_col])
    phate_operator = phate.PHATE(n_jobs=-2,knn=50)
    Y_phate = phate_operator.fit_transform(X_input)
    phate_operator.set_params(n_components=10)
    Y_phate_3d = phate_operator.transform()
    
    for i in range(10):
        df_out["phate_"+str(i)] = Y_phate_3d[:,i]
        
    knn = NearestNeighbors(n_neighbors=10000)
    phate_list = ["phate_"+str(i) for i in range(10)]
    knn.fit(df_out[phate_list].values)
    temp1 = df_out.loc[df_out.id==0]
    temp2 = knn.kneighbors(temp1[phate_list].values, return_distance=False)
    check = df_out["FJComp-PE-A"].values
    
    t1 = get_list(temp2)
    t2 = [check[i].mean() for i in t1]
    df_or[name_col] = t2

In [5]:
save_path = "../../output/human_screen/heat_map_assemble/"
path = "../../data/human data/hCB_Screering/Plate-1/"
count=0
for i in [578]:
#for i in range(426,617,2):
    count+=1
    get_assemble(path,i, fin,"screen_p1_"+str(i))
fin.to_csv(save_path+"human_screen_p1d_add.csv", index=None)

Calculating PHATE...
  Running PHATE on 67058 cells and 13 genes.
  Calculating graph and diffusion operator...
    Calculating KNN search...
    Calculated KNN search in 113.71 seconds.
    Calculating affinities...
    Calculated affinities in 16.05 seconds.
  Calculated graph and diffusion operator in 130.73 seconds.
  Calculating landmark operator...
    Calculating SVD...
    Calculated SVD in 97.05 seconds.
    Calculating KMeans...
    Calculated KMeans in 197.12 seconds.
  Calculated landmark operator in 297.92 seconds.
  Calculating optimal t...
    Automatically selected t = 23
  Calculated optimal t in 21.07 seconds.
  Calculating diffusion potential...
  Calculated diffusion potential in 1.15 seconds.
  Calculating metric MDS...
  Calculated metric MDS in 22.00 seconds.
Calculated PHATE in 472.95 seconds.
Calculating metric MDS...
Calculated metric MDS in 284.09 seconds.


In [36]:
fin.shape

(17387, 398)

In [29]:
# match all screening parameter into the reference data
from numba import njit
from sklearn.neighbors import NearestNeighbors
@njit(parallel=True)
def get_list(k):
    return [[h-17387 for h in k[x] if h>17387][:10] for x in range(k.shape[0])]

def get_assemble_v2(df , df_or, name_list):
    #df = df.loc[df["FJComp-BUV395-A"] > 200]
    df["id"] = 0

    df_out = pd.concat([df_or[dep_col+["id"]],df[dep_col+["id"]]])
    
    X_input = preprocessing.scale(df_out[dep_col])
    phate_operator = phate.PHATE(n_jobs=-2,knn=50)
    Y_phate = phate_operator.fit_transform(X_input)
    phate_operator.set_params(n_components=10)
    Y_phate_3d = phate_operator.transform()
    
    for i in range(10):
        df_out["phate_"+str(i)] = Y_phate_3d[:,i]
        
    knn = NearestNeighbors(n_neighbors=10000)
    phate_list = ["phate_"+str(i) for i in range(10)]
    knn.fit(df_out[phate_list].values)
    temp1 = df_out.loc[df_out.id!=0]
    temp2 = knn.kneighbors(temp1[phate_list].values, return_distance=False)
    #check = df_out["FJComp-PE-A"].values
    
    t1 = get_list(temp2)
    for name_col in name_list:
        check=df[name_col].values
        t2 = [check[i].mean() for i in t1]
        df_or[name_col] = t2

In [30]:
get_assemble_v2(df_or, fin, name_list)

Calculating PHATE...
  Running PHATE on 216245 cells and 13 genes.
  Calculating graph and diffusion operator...
    Calculating KNN search...
    Calculated KNN search in 825.01 seconds.
    Calculating affinities...
    Calculated affinities in 35.73 seconds.
  Calculated graph and diffusion operator in 864.45 seconds.
  Calculating landmark operator...
    Calculating SVD...
    Calculated SVD in 253.80 seconds.
    Calculating KMeans...
    Calculated KMeans in 61.69 seconds.
  Calculated landmark operator in 321.04 seconds.
  Calculating optimal t...
    Automatically selected t = 23
  Calculated optimal t in 1.34 seconds.
  Calculating diffusion potential...
  Calculated diffusion potential in 0.49 seconds.
  Calculating metric MDS...
  Calculated metric MDS in 14.32 seconds.
Calculated PHATE in 1201.68 seconds.
Calculating metric MDS...
Calculated metric MDS in 110.41 seconds.


In [31]:
fin.head()

,FSC-A,FSC-H,FSC-W,SSC-A,SSC-H,SSC-W,FJComp-APC-A,FJComp-APC-Cy7-A,FJComp-BUV395-A,FJComp-BV421-A,FJComp-BV711-A,FJComp-FITC-A,FJComp-PE-A,FJComp-PerCP-Cy5-5-A,id,screen_p1_426,screen_p1_428,screen_p1_430,screen_p1_432,screen_p1_434,screen_p1_436,screen_p1_438,screen_p1_440,screen_p1_442,screen_p1_444,screen_p1_446,screen_p1_448,screen_p1_450,screen_p1_452,screen_p1_454,screen_p1_456,screen_p1_458,screen_p1_460,screen_p1_462,screen_p1_464,screen_p1_466,screen_p1_468,screen_p1_470,screen_p1_472,screen_p1_474,screen_p1_476,screen_p1_478,screen_p1_480,screen_p1_482,screen_p1_484,screen_p1_486,screen_p1_488,screen_p1_490,screen_p1_492,screen_p1_494,screen_p1_496,screen_p1_498,screen_p1_500,screen_p1_502,screen_p1_504,screen_p1_506,screen_p1_508,screen_p1_510,screen_p1_512,screen_p1_514,screen_p1_516,screen_p1_518,screen_p1_520,screen_p1_522,screen_p1_524,screen_p1_526,screen_p1_528,screen_p1_530,screen_p1_532,screen_p1_534,screen_p1_536,screen_p1_538,screen_p1_540,screen_p1_542,screen_p1_544,screen_p1_546,screen_p1_548,screen_p1_550,screen_p1_552,screen_p1_554,screen_p1_556,screen_p1_558,screen_p1_560,screen_p1_562,screen_p1_564,screen_p1_566,screen_p1_568,screen_p1_570,screen_p1_572,screen_p1_574,screen_p1_576,screen_p1_578,screen_p1_580,screen_p1_582,screen_p1_584,screen_p1_586,screen_p1_588,screen_p1_590,screen_p1_592,screen_p1_594,screen_p1_596,screen_p1_598,screen_p1_600,screen_p1_602,screen_p1_604,screen_p1_606,screen_p1_608,screen_p1_610,screen_p1_612,screen_p1_614,screen_p1_616,screen_p2_306,screen_p2_308,screen_p2_310,screen_p2_312,screen_p2_314,screen_p2_316,screen_p2_318,screen_p2_320,screen_p2_322,screen_p2_324,screen_p2_326,screen_p2_328,screen_p2_330,screen_p2_332,screen_p2_334,screen_p2_336,screen_p2_338,screen_p2_340,screen_p2_342,screen_p2_344,screen_p2_346,screen_p2_348,screen_p2_350,screen_p2_352,screen_p2_354,screen_p2_356,screen_p2_358,screen_p2_360,screen_p2_362,screen_p2_364,screen_p2_366,screen_p2_368,screen_p2_370,screen_p2_372,screen_p2_374,screen_p2_376,screen_p2_378,screen_p2_380,screen_p2_382,screen_p2_384,screen_p2_386,screen_p2_388,screen_p2_390,screen_p2_392,screen_p2_394,screen_p2_396,screen_p2_398,screen_p2_400,screen_p2_402,screen_p2_404,screen_p2_406,screen_p2_408,screen_p2_410,screen_p2_412,screen_p2_414,screen_p2_416,screen_p2_418,screen_p2_420,screen_p2_422,screen_p2_424,screen_p2_426,screen_p2_428,screen_p2_430,screen_p2_432,screen_p2_434,screen_p2_436,screen_p2_438,screen_p2_440,screen_p2_442,screen_p2_444,screen_p2_446,screen_p2_448,screen_p2_450,screen_p2_452,screen_p2_454,screen_p2_456,screen_p2_458,screen_p2_460,screen_p2_462,screen_p2_464,screen_p2_466,screen_p2_468,screen_p2_470,screen_p2_472,screen_p2_474,screen_p2_476,screen_p2_478,screen_p2_480,screen_p2_482,screen_p2_484,screen_p2_486,screen_p2_488,screen_p2_490,screen_p2_492,screen_p2_494,screen_p2_496,screen_p3_323,screen_p3_325,screen_p3_327,screen_p3_329,screen_p3_331,screen_p3_333,screen_p3_335,screen_p3_337,screen_p3_339,screen_p3_341,screen_p3_343,screen_p3_345,screen_p3_347,screen_p3_349,screen_p3_351,screen_p3_353,screen_p3_355,screen_p3_357,screen_p3_359,screen_p3_361,screen_p3_363,screen_p3_365,screen_p3_367,screen_p3_369,screen_p3_371,screen_p3_373,screen_p3_375,screen_p3_377,screen_p3_379,screen_p3_381,screen_p3_383,screen_p3_385,screen_p3_387,screen_p3_389,screen_p3_391,screen_p3_393,screen_p3_395,screen_p3_397,screen_p3_399,screen_p3_401,screen_p3_403,screen_p3_405,screen_p3_407,screen_p3_409,screen_p3_411,screen_p3_413,screen_p3_415,screen_p3_417,screen_p3_419,screen_p3_421,screen_p3_423,screen_p3_425,screen_p3_427,screen_p3_429,screen_p3_431,screen_p3_433,screen_p3_435,screen_p3_437,screen_p3_439,screen_p3_441,screen_p3_443,screen_p3_445,screen_p3_447,screen_p3_449,screen_p3_451,screen_p3_453,screen_p3_455,screen_p3_457,screen_p3_459,screen_p3_461,screen_p3_463,screen_p3_465,screen_p3_467,screen_p3_469,screen_p3_471,screen_p3_473,screen_p3_475,screen_p3_477,screen_p3_479,screen_p3_481,screen_p3_483,screen_p3_485,s

In [32]:
df_or["id"]=0
df_t1 = df_or.loc[df_or.screen_p1_426<200]
#df_t1 = df_t1.loc[df_t1.screen_p2_376>500]
df_t1.reset_index(inplace=True)
df_t1.drop(columns=["index"], inplace=True)
fin = fin[df_t1.columns]
df_fin = pd.concat([df_t1, fin])
df_fin.shape

(127659, 398)

In [37]:
pd.crosstab(fin.id, columns="count")

col_0,count
id,
1,3474
2,7366
3,5815
4,89
5,643


In [45]:
fin_d1 = fin.loc[fin.id!=4]
X_input = preprocessing.scale(fin_d1[dep_col])
phate_operator = phate.PHATE(n_jobs=-2,knn=50)
Y_phate = phate_operator.fit_transform(X_input)
phate_operator.set_params(n_components=10)
Y_phate_3d = phate_operator.transform()
np.save(save_path + "phate10d_human_pop_d_z1.npy",Y_phate_3d)
phate_operator.set_params(n_components=3)
Y_phate_3d = phate_operator.transform()
np.save(save_path + "phate3d_human_pop_d_z1.npy",Y_phate_3d)

  Calculating PHATE...
    Running PHATE on 17298 cells and 397 genes.
    Calculating graph and diffusion operator...
      Calculating PCA...
      Calculated PCA in 0.71 seconds.
      Calculating KNN search...
      Calculated KNN search in 9.89 seconds.
      Calculating affinities...
      Calculated affinities in 2.67 seconds.
    Calculated graph and diffusion operator in 13.40 seconds.
    Calculating landmark operator...
      Calculating SVD...
      Calculated SVD in 6.48 seconds.
      Calculating KMeans...
      Calculated KMeans in 26.11 seconds.
    Calculated landmark operator in 33.89 seconds.
    Calculating optimal t...
      Automatically selected t = 45
    Calculated optimal t in 1.31 seconds.
    Calculating diffusion potential...
    Calculated diffusion potential in 0.53 seconds.
    Calculating metric MDS...
  Calculated metric MDS in 19.96 seconds.
Calculated PHATE in 69.10 seconds.
Calculating metric MDS...
Calculated metric MDS in 274.06 seconds.
Calculati

In [18]:
for i in df_fin.columns:
    if df_fin[i].isnull().sum()>0:
        print(i)
        print(df_fin[i].isnull().sum())

In [24]:
X_input = preprocessing.scale(df_fin[dep_col])
phate_operator = phate.PHATE(n_jobs=-2,knn=50)
Y_phate = phate_operator.fit_transform(X_input)
phate_operator.set_params(n_components=10)
Y_phate_3d = phate_operator.transform()
np.save(save_path + "phate10d_human_pop_d.npy",Y_phate_3d)
phate_operator.set_params(n_components=3)
Y_phate_3d = phate_operator.transform()
np.save(save_path + "phate3d_human_pop_d.npy",Y_phate_3d)

Calculating PHATE...
  Running PHATE on 127659 cells and 397 genes.
  Calculating graph and diffusion operator...
    Calculating PCA...
    Calculated PCA in 6.99 seconds.
    Calculating KNN search...
    Calculated KNN search in 351.69 seconds.
    Calculating affinities...
    Calculated affinities in 22.14 seconds.
  Calculated graph and diffusion operator in 382.89 seconds.
  Calculating landmark operator...
    Calculating SVD...
    Calculated SVD in 119.98 seconds.
    Calculating KMeans...
    Calculated KMeans in 53.03 seconds.
  Calculated landmark operator in 176.59 seconds.
  Calculating optimal t...
    Automatically selected t = 25
  Calculated optimal t in 1.44 seconds.
  Calculating diffusion potential...
  Calculated diffusion potential in 0.51 seconds.
  Calculating metric MDS...
  Calculated metric MDS in 15.20 seconds.
Calculated PHATE in 576.66 seconds.
Calculating metric MDS...
Calculated metric MDS in 160.86 seconds.
Calculating metric MDS...
Calculated metric 

In [29]:
save_path = "../../output/human_screen/heat_map_assemble/"

Y_phate_3d = np.load(save_path + "phate3d_15072020.npy")

temp1 = {}
temp1["x"] = Y_phate_3d[:,0]
temp1["y"] = Y_phate_3d[:,1]
temp1["z"] = Y_phate_3d[:,2]
temp1["cid"] = df_fin.id
#temp1["cid"] = np.concatenate([df_t1.id.values, df_in.id.values])
temp1 = pd.DataFrame(temp1)
from sklearn.preprocessing import MinMaxScaler
import json
scaler = MinMaxScaler( feature_range=( -100, 100 ))
temp1[[ 'x', 'y', 'z' ]] = scaler.fit_transform( temp1[[ 'x', 'y', 'z' ]])

d_json = { 'points': json.loads( temp1.astype( str ).to_json( None, orient= 'records' )) }
json.dump( d_json, open("../../data/newData/data.json", 'w' ))

In [46]:
fin_d1["id1"] = [0 if i==1 else 1 if i==2 else 2 if i==3 else 3 for i in fin_d1.id]

In [47]:
save_path = "../../output/human_screen/heat_map_assemble/"

Y_phate_3d = np.load(save_path + "phate3d_human_pop_d_z1.npy")

temp1 = {}
temp1["x"] = Y_phate_3d[:,0]
temp1["y"] = Y_phate_3d[:,1]
temp1["z"] = Y_phate_3d[:,2]
temp1["cid"] = fin_d1.id1
#temp1["cid"] = np.concatenate([df_t1.id.values, df_in.id.values])
temp1 = pd.DataFrame(temp1)
from sklearn.preprocessing import MinMaxScaler
import json
scaler = MinMaxScaler( feature_range=( -100, 100 ))
temp1[[ 'x', 'y', 'z' ]] = scaler.fit_transform( temp1[[ 'x', 'y', 'z' ]])

d_json = { 'points': json.loads( temp1.astype( str ).to_json( None, orient= 'records' )) }
json.dump( d_json, open("../../data/newData/data.json", 'w' ))

In [ ]:
temp = pd.read_csv("temp_v5.csv")
list(temp.columns)

In [13]:
from matplotlib import cm
import matplotlib as mpl
red = cm.get_cmap('Reds', 9)
blue = cm.get_cmap('Blues', 9)
cmap_t = mpl.colors.ListedColormap(np.concatenate([blue([0.75]),red(np.linspace(0, 1, 9))]))
fut = Flow_Util()

save_path = "../../output/human_screen/Preliminary_results/Plate_4/"
np_path = "../../output/human_screen/plate_4/"
or_path = "../../data/human data/hCB_Screering/Plate-4/"
dirlist = os.listdir(save_path)
for i in [904,905]:
    print(i)
    tempdir = [k for k in dirlist if str(i) in k]
    print(tempdir[0])
    df_list = fut.read_fcs(or_path+str(i)+"_down.fcs")
    df_list["check"] = [0 if i<200 else 1 if i<300 else 2 if i<500 else 3 if i<1000 else 4 if i<2000 
                        else 5 if i<5000 else 6 if i<10000 else 7 if i<50000 else 8 if i<100000 
                        else 9 for i in df_list['FJComp-PE-A']]
    Y_phate_3d = np.load(np_path + "phate3d_"+str(i)+".npy")
    temp = [np.concatenate([Y_phate_3d[:,0],[0]*10]),np.concatenate([Y_phate_3d[:,1],[0]*10]),np.concatenate([Y_phate_3d[:,2],[0]*10])]
    temp1 = np.concatenate([df_list.check.values,[0,1,2,3,4,5,6,7,8,9]])
    scprep.plot.rotate_scatter3d(temp,c=temp1,filename = save_path+tempdir[0]+"/"+str(i)+"_pe.mp4", figsize=(15,10), cmap=cmap_t,ticks=False, label_prefix="PHATE")
    plt.close("all")

904
904
905
905


In [14]:
for i in [904,905]:
    print(i)
    tempdir = [k for k in dirlist if str(i) in k]
    print(tempdir[0])
    df_list = fut.read_fcs(or_path+str(i)+"_down.fcs")
    df_list["check"] = [0 if i<200 else 1 if i<300 else 2 if i<500 else 3 if i<1000 else 4 if i<2000 
                        else 5 if i<5000 else 6 if i<10000 else 7 if i<50000 else 8 if i<100000 
                        else 9 for i in df_list['FJComp-BUV395-A']]
    Y_phate_3d = np.load(np_path + "phate3d_"+str(i)+".npy")
    temp = [np.concatenate([Y_phate_3d[:,0],[0]*10]),np.concatenate([Y_phate_3d[:,1],[0]*10]),np.concatenate([Y_phate_3d[:,2],[0]*10])]
    temp1 = np.concatenate([df_list.check.values,[0,1,2,3,4,5,6,7,8,9]])
    scprep.plot.rotate_scatter3d(temp,c=temp1,filename = save_path+tempdir[0]+"/"+str(i)+"_cd34.mp4", figsize=(15,10), cmap=cmap_t,ticks=False, label_prefix="PHATE")
    plt.close("all")

904
904
905
905
